# 注意力机制

## 1、注意力机制简介

在注意力机制中，Q 、K 、V 分别代表 Query（查询） 、Key（键） 和 Value（值）  
1、Q 是用来表示当前需要关注的内容或查询的目标。可以将其理解为一个“问题”或“需求”，它用于与其他内容进行匹配，以找到相关的信息；  
2、K 是用来表示被查询的内容或可供匹配的信息。可以将其理解为一个“标识符”或“索引”，它与 Q 进行相似度计算，从而判断两者的相关性； 
3、V 是实际的内容或信息载体，包含了具体的语义信息。当 Q 和 K 计算出相似度后，通过加权求和的方式从 V 中提取相关信息。  
  
Q 和 K 的点积结果 被称为“注意力分数”（attention score），它表示当前查询（Query）对某个键（Key）的关注程度，注意力分数越高，说明该键对应的值（Value）对当前查询更重要。

## 2、自注意力机制简单示例

In [18]:
import torch
import torch.nn.functional as F
B, T, C = 4, 8, 2
v = torch.randn(B, T, C)

tril = torch.tril(torch.ones((T, T)))
wei = torch.zeros((T, T))
wei_mask = wei.masked_fill(tril == 0, float('-inf'))
wei_softmax = F.softmax(wei_mask, dim=-1)
out = wei_softmax @ v

In [8]:
tril, tril.shape

(tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1.]]),
 torch.Size([8, 8]))

In [13]:
wei_mask, wei_mask.shape

(tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
         [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
         [0., 0., 0., 0., -inf, -inf, -inf, -inf],
         [0., 0., 0., 0., 0., -inf, -inf, -inf],
         [0., 0., 0., 0., 0., 0., -inf, -inf],
         [0., 0., 0., 0., 0., 0., 0., -inf],
         [0., 0., 0., 0., 0., 0., 0., 0.]]),
 torch.Size([8, 8]))

In [14]:
wei_softmax, wei_softmax.shape

(tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
         [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
         [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
         [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]]),
 torch.Size([8, 8]))

out = wei_softmax @ v：(B, T, T) @ (B, T, C) -> (B, T, C)  
完成的效果即为对过去的信息进行加权求和，得到当前时刻的输出

In [24]:
print(v[0][0][0] == out[0][0][0])  # True
print((v[0][0][0] + v[0][1][0]) / 2 == out[0][1][0])   # True

tensor(True)
tensor(True)


In [20]:
v[0], out[0]

(tensor([[ 1.2549, -0.3037],
         [-0.4198,  1.5528],
         [ 1.0640, -0.3529],
         [-1.7614,  0.2654],
         [ 1.2936, -1.0602],
         [-1.3749, -0.0089],
         [ 1.3363, -0.9032],
         [ 0.3539, -1.3199]]),
 tensor([[ 1.2549, -0.3037],
         [ 0.4175,  0.6245],
         [ 0.6330,  0.2987],
         [ 0.0344,  0.2904],
         [ 0.2863,  0.0203],
         [ 0.0094,  0.0154],
         [ 0.1989, -0.1158],
         [ 0.2183, -0.2663]]))

## 3、自注意力机制在LLM中的实现

In [ ]:
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
import math

@dataclass
class GPTConfig:
    block_size: int = 1024   # 句子长度
    n_head: int = 12    # 注意力头数
    n_embd: int = 768   # 每个词的向量维度，即隐藏层维度

class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        # regularization
        self.n_head = config.n_head
        self.n_embd = config.n_embd

        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size)).view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size()

        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        '''
        att即相当于第一节当中的wei_softmax，只不过初始信息为q与k的相乘
        '''
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)

        # 使用该行代替上面的注意力求解过程的四行，即可应用flash-attention加速
        # y = F.scaled_dot_product_attention(q, k, v,is_causal=True)

        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.c_proj(y)
        return y
    
if __name__ == '__main__':
    config = GPTConfig(block_size=8, n_head=2, n_embd=4)
    model = CausalSelfAttention(config)
    x = torch.rand(2, 8, 4)
    y = model(x)
    print(y.shape)

torch.Size([2, 8, 4])
